In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, zscore
from natsort import natsorted
import plotly.express as px

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Saved data information:
The data for rotated place fields (results come from 04_rotate_placefields.py) are saved in a consistent naming format. Take, for example, A1_A5_pearson_place. The first grouping (A1) means that the reference for the analysis was the first day in A. The second grouping (A5) means that the reference was compared to every day in A up until day 5. Pearson tells us that the correlations are pearson correlations, and place tells us that only place cells were included.

In [5]:
## Settings
dpath = '../../../MultiCon_Imaging/MultiCon_Imaging2/output/'
fig_path = '../../../MultiCon_Imaging/MultiCon_Imaging2/intermediate_figures/'
mouse_list = ['mc23', 'mc26']
n_colors = 100
colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])

In [101]:
## Compare day 1 to days 1 through 5 in A for both mice
outputA = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/A1_A5_pearson_place.feat'))
        outputA = pd.concat([outputA, loop_data])
mouse_data = outputA.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=5, 
                               width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(1, 6)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color='turquoise',
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='turquoise',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='turquoise',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'A1_A5_pearson_place.png'))

In [102]:
## Compare day 6 to days 6 through 10 in B for both mice
outputB = pd.read_feather(pjoin(dpath, 'rotated_placefields/mc26/B1_B5_pearson_place.feat'))
mouse_data = outputB.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=5, 
                               width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(6, 11)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color='orange',
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='orange',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='orange',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'B1_B5_pearson_place.png'))

In [103]:
## Compare day 11 to days 11 through 15 in C for both mice
outputC = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/C1_C5_pearson_place.feat'))
        outputC = pd.concat([outputC, loop_data])
mouse_data = outputC.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=5, 
                               width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(11, 16)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color='darkorchid',
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='darkorchid',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='darkorchid',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'C1_C5_pearson_place.png'))

In [104]:
## Compare day 16 to days 16 through 20 in B for mc26, since mc23 had imaging issues on day 16
outputD = pd.DataFrame()
for mouse in mouse_list:
    if mouse == 'mc23':
          pass
    else:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/D1_D5_pearson_place.feat'))
        outputD = pd.concat([outputD, loop_data])
mouse_data = outputD.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=4, 
                               width=2000, height=640, specs=specs, titles=['Day 16', 'Day 17', 'Day 18', 'Day 20'])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color='darkgreen',
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='darkgreen',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color='darkgreen',
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'D1_D5_pearson_place.png'))

In [143]:
## Compare the last day in A to days 6-10 in B
outputAB = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/A5_B5_pearson_place.feat'))
        outputAB = pd.concat([outputAB, loop_data])
mouse_data = outputAB.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=6, 
                               width=2500, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(5, 11)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        if idx == 0:
                bar_color = 'midnightblue' 
        else:
                bar_color = 'orange'
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color=bar_color,
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'A5_B5_pearson_place.png'))

In [144]:
## Compare the last day in B to days 11-15 in C
outputBC = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/B5_C5_pearson_place.feat'))
        outputBC = pd.concat([outputBC, loop_data])
mouse_data = outputBC.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=6, 
                               width=2500, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(10, 16)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        if idx == 0:
                bar_color = 'orange'
        else:
                bar_color = 'darkorchid'
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color=bar_color,
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'B5_C5_pearson_place.png'))

In [145]:
## Compare the last day in C to days 16-20 in D
outputCD = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/C5_D5_pearson_place.feat'))
        outputCD = pd.concat([outputCD, loop_data])
mouse_data = outputCD.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', font_size=22, rows=1, columns=6, 
                               width=2500, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(15, 21)])
for idx, session in enumerate(natsorted(np.unique(fig_data['session']))):
        if idx == 0:
                bar_color = 'darkorchid' 
        else:
                bar_color = 'darkgreen'
        fig.add_trace(go.Barpolar(
            r=fig_data['r']['mean'][fig_data['session'] == session],
            theta=fig_data['rotation'][fig_data['session'] == session],
            marker_color=bar_color,
            marker_line_color='black',
            marker_line_width=1,
            opacity=0.8,
            showlegend=False
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] + fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='upper'
        ), row=1, col=idx + 1)
        fig.add_trace(go.Barpolar(
                r=fig_data['r']['mean'][fig_data['session'] == session] - fig_data['r']['sem'][fig_data['session'] == session],
                theta=fig_data['rotation'][fig_data['session'] == session],
                marker_color=bar_color,
                marker_line_color='black',
                marker_line_width=0,
                opacity=0.3,
                showlegend=False,
                name='lower'
        ), row=1, col=idx + 1)
    
fig.update_polars(
        radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
        angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
        barmode='overlay'
    )
fig.show()
fig.write_image(pjoin(fig_path, 'C5_D5_pearson_place.png'))

### Correlations within a context, but reference is from Day 5

In [135]:
## Compare the last day in A to the first 4 days in A
outputA = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/A5_A5_pearson_place.feat'))
        outputA = pd.concat([outputA, loop_data])
mouse_data = outputA.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.create_polar_plots(fig_data=fig_data, bar_color='turquoise', data_col='r', error_col='sem',
                         x_title='', y_title='', font_size=22, rows=1, columns=5, 
                         width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(1, 6)])
fig.show()
fig.write_image(pjoin(fig_path, 'A5_A5_pearson_place.png'))

In [136]:
## Compare the last day in B to the first 4 days in B
outputB = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/B5_B5_pearson_place.feat'))
        outputB = pd.concat([outputB, loop_data])
mouse_data = outputB.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.create_polar_plots(fig_data=fig_data, bar_color='orange', data_col='r', error_col='sem',
                         x_title='', y_title='', font_size=22, rows=1, columns=5, 
                         width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(6, 11)])
fig.show()
fig.write_image(pjoin(fig_path, 'B5_B5_pearson_place.png'))

In [137]:
## Compare the last day in C to the first 4 days in C
outputC = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/C5_C5_pearson_place.feat'))
        outputC = pd.concat([outputC, loop_data])
mouse_data = outputC.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.create_polar_plots(fig_data=fig_data, bar_color='darkorchid', data_col='r', error_col='sem',
                         x_title='', y_title='', font_size=22, rows=1, columns=5, 
                         width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(11, 16)])
fig.show()
fig.write_image(pjoin(fig_path, 'C5_C5_pearson_place.png'))

In [138]:
## Compare the last day in D to the first 4 days in D
outputD = pd.DataFrame()
for mouse in mouse_list:
        loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/D5_D5_pearson_place.feat'))
        outputD = pd.concat([outputD, loop_data])
mouse_data = outputD.groupby(['mouse', 'session', 'rotation'], as_index=False).agg({'r': 'mean'})
fig_data = mouse_data.groupby(['session', 'rotation'], as_index=False).agg({'r': ['mean', 'sem']})
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.create_polar_plots(fig_data=fig_data, bar_color='darkgreen', data_col='r', error_col='sem',
                         x_title='', y_title='', font_size=22, rows=1, columns=5, 
                         width=2300, height=600, specs=specs, titles=[f'Day {x}' for x in np.arange(16, 21)])
fig.show()
fig.write_image(pjoin(fig_path, 'D5_D5_pearson_place.png'))

### Look at individual cells

In [148]:
ind_cells = pd.DataFrame()
for mouse in mouse_list:
    loop_data = pd.read_feather(pjoin(dpath, f'rotated_placefields/{mouse}/A5_A5_pearson_place.feat'))
    ind_cells = pd.concat([ind_cells, loop_data])

In [188]:
np.unique(sess_data['ref_unit_id']).shape

(30,)

In [209]:
rot_data = ind_cells.groupby(['mouse', 'session', 'rotation', 'ref_unit_id'], as_index=False).agg({'r': ['mean', 'sem']})
rot_data = rot_data[rot_data['mouse'] == 'mc23']
specs = [[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
fig = pf.custom_graph_template(x_title='', y_title='', titles=[f'Day {x}' for x in np.arange(1, 6)], 
                               width=2300, height=600, specs=specs, rows=1, columns=5)
for idx, session in enumerate(natsorted(np.unique(rot_data['session']))):
    sess_data = rot_data[rot_data['session'] == session]
    for unit in np.unique(sess_data['ref_unit_id']):
        unit_data = sess_data[sess_data['ref_unit_id'] == unit]
        if unit_data.empty:
            pass
        else:
            fig.add_trace(go.Scatterpolar(
                r=unit_data['r']['mean'],
                theta=unit_data['rotation'],
                marker_color=colors[unit],
                marker_line_color='black',
                marker_line_width=1,
                opacity=0.5,
                mode='markers',
                showlegend=False,
                name=str(unit)
            ), row=1, col=idx + 1)
fig.update_polars(
            radialaxis = dict(showticklabels=False, ticks='', angle=180, tickangle=-180, range=[-0.2, 1], showgrid=True, gridcolor='darkgrey', dtick=0.2),
            angularaxis = dict(showticklabels=True, ticks='', showgrid=True, gridcolor='darkgrey'),
            barmode='overlay'
        )
fig.show()